In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-96r5vxu3
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-96r5vxu3
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import clip
from PIL import Image
import os
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 167MiB/s]


In [5]:
!unzip "/content/drive/MyDrive/testdataset.zip"

Archive:  /content/drive/MyDrive/testdataset.zip
   creating: testdata/
  inflating: testdata/  001.jpg      
  inflating: testdata/  002.jpg      
  inflating: testdata/  003.jpg      
  inflating: testdata/  004.jpg      
  inflating: testdata/  005.jpg      
  inflating: testdata/  006.jpg      
  inflating: testdata/  007.jpg      
  inflating: testdata/  008.jpg      
  inflating: testdata/  009.jpg      
  inflating: testdata/  010.jpg      
  inflating: testdata/  011.jpg      
  inflating: testdata/  012.jpg      
  inflating: testdata/  013.jpg      
  inflating: testdata/  014.jpg      
  inflating: testdata/  015.jpg      
  inflating: testdata/  016.jpg      
  inflating: testdata/  017.jpg      
  inflating: testdata/  018.jpg      
  inflating: testdata/  019.jpg      
  inflating: testdata/  020.jpg      
  inflating: testdata/  021.jpg      
  inflating: testdata/  022.jpg      
  inflating: testdata/  023.jpg      
  inflating: testdata/  024.jpg      
  inflating: tes

In [6]:
image_dir = '/content/testdata/'  # Update this path
image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.jpg')])

# Batch processing for efficiency
batch_size = 32
image_embeddings = []

for i in range(0, len(image_files), batch_size):
    batch_paths = image_files[i:i+batch_size]
    batch_images = []
    for path in batch_paths:
        image = Image.open(path).convert("RGB")
        batch_images.append(preprocess(image))
    batch_images = torch.stack(batch_images).to(device)
    with torch.no_grad():
        batch_features = model.encode_image(batch_images)
    image_embeddings.append(batch_features.cpu())

image_embeddings = torch.cat(image_embeddings, dim=0).to(device)
image_embeddings /= image_embeddings.norm(dim=-1, keepdim=True)

In [9]:
#caption_dir = '/content/VITGPT2_captions.txt/'  # Update this path
#caption_files = [f for f in os.listdir(caption_dir) if f.endswith('.txt')]

caption_dir = '/content/'  # Update this to the actual directory containing the caption files
caption_files = [f for f in os.listdir(caption_dir) if f.endswith('.txt')]

# ... rest of the code ...

for caption_file in caption_files:
    # Read captions
    with open(os.path.join(caption_dir, caption_file), 'r') as f:
        captions = [line.strip() for line in f.readlines()]

    # Process text embeddings
    text_inputs = clip.tokenize(captions).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Calculate scores
    min_len = min(image_embeddings.shape[0], text_features.shape[0])
similarity = (image_embeddings[:min_len] * text_features[:min_len]).sum(dim=1)
clip_scores = 2.5 * torch.clamp(similarity, min=0)
average_score = clip_scores.mean().item()
print(f"CLIPScore for {caption_file}: {average_score:.4f}")

CLIPScore for VITGPT2_captions.txt: 0.7061


In [10]:
!unzip "/content/captions.zip"

Archive:  /content/captions.zip
   creating: captions/
  inflating: captions/BLIP2_captions.txt  
  inflating: captions/Fuyu_8B_captions.txt  
  inflating: captions/GIT_captions.txt  
  inflating: captions/kosmos-2_captions.txt  
  inflating: captions/LLaVA_captions.txt  
  inflating: captions/moondream2_captions.txt  
  inflating: captions/OFA_captions.txt  
  inflating: captions/VITGPT2_captions.txt  


In [13]:
caption_dir = '/content/captions/'  # Update this path
caption_files = [f for f in os.listdir(caption_dir) if f.endswith('.txt')]


for caption_file in caption_files:
    # Read captions
    with open(os.path.join(caption_dir, caption_file), 'r') as f:
        captions = [line.strip() for line in f.readlines()]

    # Process text embeddings
    # Process text embeddings
#Truncate the captions to a maximum of 77 tokens
    text_inputs = clip.tokenize(captions, truncate=True).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Calculate scores
    min_len = min(image_embeddings.shape[0], text_features.shape[0]) # This line was incorrectly indented. Fixed.
    similarity = (image_embeddings[:min_len] * text_features[:min_len]).sum(dim=1)
    clip_scores = 2.5 * torch.clamp(similarity, min=0)
    average_score = clip_scores.mean().item()
    print(f"CLIPScore for {caption_file}: {average_score:.4f}")

CLIPScore for BLIP2_captions.txt: 0.7183
CLIPScore for LLaVA_captions.txt: 0.7568
CLIPScore for moondream2_captions.txt: 0.4558
CLIPScore for GIT_captions.txt: 0.7153
CLIPScore for OFA_captions.txt: 0.4702
CLIPScore for VITGPT2_captions.txt: 0.7061
CLIPScore for kosmos-2_captions.txt: 0.7446
CLIPScore for Fuyu_8B_captions.txt: 0.7070
